In [7]:
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time
from os import system, name, path
from bs4 import BeautifulSoup
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

import os
if not os.path.exists("./raw"):
    os.makedirs("./raw")

In [8]:
# Global Constant
num_posts_per_cat = 500
category = pd.read_csv('./hashtag_list.csv')

In [9]:
## Chrome
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito")
driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(), chrome_options=chrome_options)

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 97.0.4692
[WDM] - Get LATEST driver version for 97.0.4692
[WDM] - Driver [/home/victor/.wdm/drivers/chromedriver/linux64/97.0.4692.71/chromedriver] found in cache


In [10]:
# Login
def login(user, pw):
    try:
        driver.get('https://www.facebook.com')
        username = driver.find_element_by_id("email")
        password = driver.find_element_by_id("pass")
        username.send_keys(user)
        password.send_keys(pw)
        driver.find_elements_by_name("login")[0].click()
    except:
        print('Login Failed...')
# Get Hashtag
def getPosts(hashtag, n=100, search_list=None):
    try:
        result = []
        if not search_list:
            search_list = [hashtag]
        for tag in search_list:
            if len(result) >= n:
                break
            print(f"Using keyword - {tag}")
            driver.get(f'https://www.facebook.com/hashtag/{tag}')
            stag_count = 0
            cur_found = 0
            retry_count = 0
            # Scroll until get n posts
            while cur_found < n - len(result) + 20:
                if cur_found == len(driver.find_elements_by_xpath("//div[@role='article']")):
                    stag_count += 1
                else:
                    stag_count = 0
                if stag_count == 50:
                    if retry_count >= 3:
                        # Stuck
                        break
                    driver.refresh()
                    stag_count = 0
                    retry_count += 1
                    continue
                print(f'Current number of posts: {cur_found}...', end='\r')
                driver.find_element_by_tag_name('html').send_keys(Keys.END)
                time.sleep(1)
                cur_found = len(driver.find_elements_by_xpath("//div[@role='article']"))
            cur_found = len(driver.find_elements_by_xpath("//div[@role='article']"))
            print(f'\nTotal number of posts found by using keyword {tag}: {cur_found}')
            # Click see more
            for i in range(cur_found):
                print(f"Clicking 'See more' of posts...{i+1}", end="\r")
                posts = driver.find_elements_by_xpath("//div[@role='article']")
                if len(posts) <= i:
                    break
                p = posts[i]
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", p)
                time.sleep(1)
                try:
                    if 'See more' in p.text:
                        seeMoreBtn = wait(p, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[contains(text(), 'See more')]")))
                        driver.execute_script("arguments[0].scrollIntoView({block: 'end'});", seeMoreBtn)
                        time.sleep(1)
                        seeMoreBtn.click()
                        time.sleep(1)
                except TimeoutException:
                    time.sleep(1)
                result.append(p.text)
            print("")
        if len(result) < n:
            raise Exception('Not enough posts after searching all tags')
        return result[:n]
    except Exception as e:
        print('')
        print(e)
        print('Search hashtag failed...')
        return None

In [11]:
total = []
failed_tag = []
for index, row in category.iterrows():
    data = pd.DataFrame()
    search_list = row['SearchList']
    tag = row['Hashtag']
    print(f"Getting hashtag - {tag} ({index+1}/{category.shape[0]})")
    # check if file already exists and having enough
    if path.exists(f"./{tag}.csv"):
        d = pd.read_csv(f"./{tag}.csv")
        if d.shape[0] >= num_posts_per_cat:
            print("Skipped")
            continue
    # check if nan
    if search_list != search_list:
        search_list = [tag]
    else:
        search_list = [tag] + search_list.split(',')
        search_list = list(dict.fromkeys(search_list).keys())
    posts = getPosts(tag, num_posts_per_cat, search_list)
    if posts == None or len(posts) == 0:
        failed_tag.append(tag)
        continue
    data['content'] = posts
    data['tag'] = tag
    data['group'] = row['Group']
    total.append(data)
    data.to_csv(f"./raw/{tag}.csv", index=False)

Getting hashtag - restaurant (1/1)
Using keyword - restaurant
Current number of posts: 132...
Total number of posts found by using keyword restaurant: 138
Clicking 'See more' of posts...138
Using keyword - restaurants
Current number of posts: 248...
Total number of posts found by using keyword restaurants: 253
Clicking 'See more' of posts...252

Not enough posts after searching all tags
Search hashtag failed...


In [12]:
failed_tag

['restaurant']